In [1]:
from utils.email_utils import send_email


In [2]:
send_email(to="sebastian@skills.tech",subject="test",content="Template corre de testing")


'Email sent successfully'

In [3]:
from supabase import create_client
from decouple import config

supabase = create_client(config("SUPABASE_URL"), config("SUPABASE_KEY"))


In [4]:
usuarios_prueba = [
    {
        "user_email": "sebastian@skills.tech",
        "reminder_type": "Learning Progress Check-In",
        "status": "pending",
        "scheduled_date": "2023-01-01T10:00:00Z",
        "content": "Just a quick reminder to check your progress on this week's topic...",
        "sent_date": None
    },
    {
        "user_email": "zuriel@skills.tech",
        "reminder_type": "Encouragement and Motivation",
        "status": "pending",
        "scheduled_date": "2023-01-02T10:00:00Z",
        "content": "You're doing an amazing job! Remember, every question you ask...",
        "sent_date": None
    },
    {
        "user_email": "sebastian@skills.tech",
        "reminder_type": "Learning Progress Check-In",
        "status": "pending",
        "scheduled_date": "2024-01-03T10:00:00Z",
        "content": "Just a quick reminder to check your progress on this week's topic...",
        "sent_date": None
    },
    # ... más usuarios de prueba ...
]

for usuario in usuarios_prueba:
    response = supabase.table("reminders_tb").insert(usuario).execute()
    try:
        if response.error:
            print(f"Error al insertar: {response.error}")
        else:
            print(f"Usuario insertado: {usuario['user_email']}")
    except:
        print("No se pudo cargar los datos ficticios")
        

2024-01-07 23:12:12,334:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"
2024-01-07 23:12:12,452:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"


No se pudo cargar los datos ficticios
No se pudo cargar los datos ficticios


2024-01-07 23:12:12,692:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"


No se pudo cargar los datos ficticios


In [27]:
from utils.email_utils import Learning_process
import datetime
import pytz  # Asegúrate de tener instalado pytz
current_date = datetime.datetime.now(datetime.timezone.utc)


In [22]:
recordatorios = supabase.table("reminders_tb").select("*").eq("reminder_type", "Learning Progress Check-In").eq("status", "pending").execute().data

for recordatorio in recordatorios:
    print(recordatorio)
    #send_email(recordatorio["user_email"], "Learning Progress Check-In", Learning_process)

    # Actualizar el estado del recordatorio en la base de datos
    supabase.table("reminders_tb").update({"status": "sent"}).eq("id", recordatorio["id"]).execute()

2024-01-08 00:15:46,541:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&reminder_type=eq.Learning%20Progress%20Check-In&status=eq.pending "HTTP/1.1 200 OK"


Conexion con members y creacion de datos en supabase reminders_tb

In [29]:
# Obtener todos los usuarios de members_tb
members = supabase.table("members_tb").select("*").execute().data
for member in members:
    user_email = member["email"]
    user_name = member["name"]  # Obtener el nombre del usuario


    # Verificar si el usuario ya tiene un recordatorio en reminders_tb
    existing_reminder = supabase.table("reminders_tb").select("*").eq("user_email", user_email).eq("reminder_type", "Welcome").execute().data

    if not existing_reminder:
        # Enviar correo de bienvenida
        welcome_content = "Bienvenido a nuestra plataforma. ¡Estamos emocionados de tenerte aquí!"
        send_email(to=user_email, subject="Bienvenido", content=welcome_content)

        # Crear un registro de recordatorio para el nuevo usuario
        new_reminder = {
            "user_email": user_email,
            "name":user_name,
            "reminder_type": "Welcome",
            "status": "sent",
            "scheduled_date": None,  # No aplica para correos de bienvenida
            "content": "Bienvenido a nuestra plataforma.",
            "sent_date": current_date.isoformat()
        }
        supabase.table("reminders_tb").insert(new_reminder).execute()

        print(f"Correo de bienvenida enviado a {user_email} y recordatorio creado.")

2024-01-08 00:21:01,266:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/members_tb?select=%2A "HTTP/1.1 200 OK"
2024-01-08 00:21:01,365:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.lc%40luiszorrilla.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 00:21:01,471:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.test%40test.test&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 00:21:01,571:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.sebastian%40skills.tech&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 00:21:02,836:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"
2024-01-08 00:21:02,937:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_

Correo de bienvenida enviado a sebastian@skills.tech y recordatorio creado.


2024-01-08 00:21:03,042:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.skills.demo%40newyorklife.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"


Envio de correos con html

In [8]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from decouple import config

email_user_cred=config("username")
email_user_pass=config("password")

def send_html_email(to, subject, html_content):
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = "arturo@skills.tech"  # Cambiar a tu dirección de correo
    msg['To'] = to

    # Parte HTML del mensaje
    html_part = MIMEText(html_content, 'html')
    msg.attach(html_part)

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
            server.login(email_user_cred, email_user_pass)  # Cambiar a tus credenciales
            server.send_message(msg)
        print(f"Correo enviado con éxito a {to}")
    except Exception as e:
        print(f"Error al enviar correo a {to}: {e}")


In [17]:
with open("email_templates/Learning_check_in.html", "r") as file:
        html_content = file.read()

html_content = html_content.replace("[Nombre del Usuario]", user_name)
send_html_email("sebastian@skills.tech", "Bienvenido a SkillsAI", html_content)

Correo enviado con éxito a sebastian@skills.tech


Correo de Learning check-in una semana despues

In [45]:
import datetime
import pytz  # Asegúrate de tener instalado pytz

current_date = datetime.datetime.now(datetime.timezone.utc)

users = supabase.table("members_tb").select("*").execute().data

for user in users:
    user_email = user["email"]

    # Buscar el recordatorio de bienvenida
    welcome_reminder = supabase.table("reminders_tb").select("*").eq("user_email", user_email).eq("reminder_type", "Welcome").execute().data

    if welcome_reminder:
        welcome_date = welcome_reminder[0]["sent_date"]
        if welcome_date:
            welcome_date = datetime.datetime.fromisoformat(welcome_date)
            if not welcome_date.tzinfo:
                welcome_date = pytz.utc.localize(welcome_date)

            scheduled_date = welcome_date + datetime.timedelta(days=7)

            # Buscar el recordatorio "Learning Progress Check-In"
            progress_checkin = supabase.table("reminders_tb").select("*").eq("user_email", user_email).eq("reminder_type", "Learning Progress Check-In").execute().data
            print(progress_checkin,current_date >= scheduled_date)
            if not progress_checkin:
                # Crear un registro para el nuevo recordatorio programado
                new_recordatorio = {
                    "user_email": user_email,
                    "user_name":user_name,
                    "reminder_type": "Learning Progress Check-In",
                    "status": "pending",
                    "scheduled_date": scheduled_date.isoformat(),
                    "content": "Learning Progress Check-In",
                    "sent_date": None
                }
                supabase.table("reminders_tb").insert(new_recordatorio).execute()
                print(f"Se programó email 'Learning Progress Check-In' para {user_email}")

            elif progress_checkin[0]["status"] == "pending" and current_date >= scheduled_date:
                # Enviar correo "Learning Progress Check-In"
                with open("email_templates/Learning_check_in.html", "r") as file:
                    html_content = file.read()

                    html_content = html_content.replace("[Nombre del Usuario]", user_name)
                    send_html_email(user_email, "Learning Progress Check-In", html_content)
                # Actualizar el estado del recordatorio
                supabase.table("reminders_tb").update({"status": "sent", "sent_date": current_date.isoformat()}).eq("id", progress_checkin[0]["id"]).execute()
                print(f"Se mando email 'Learning Progress Check-In' a {user_email}")



2024-01-08 01:01:07,220:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/members_tb?select=%2A "HTTP/1.1 200 OK"
2024-01-08 01:01:07,315:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.lc%40luiszorrilla.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:07,405:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.lc%40luiszorrilla.com&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"
2024-01-08 01:01:07,500:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.test%40test.test&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:07,597:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.test%40test.test&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"


[{'id': 57, 'user_email': 'lc@luiszorrilla.com', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T02:31:17+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False
[{'id': 58, 'user_email': 'test@test.test', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T06:31:13+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False


2024-01-08 01:01:07,690:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.sebastian%40skills.tech&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:07,783:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.sebastian%40skills.tech&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"
2024-01-08 01:01:07,877:INFO - HTTP Request: POST https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb "HTTP/1.1 201 Created"
2024-01-08 01:01:07,972:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.luis%40skills.tech&reminder_type=eq.Welcome "HTTP/1.1 200 OK"


[] False
Se programó email 'Learning Progress Check-In' para sebastian@skills.tech


2024-01-08 01:01:08,062:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.luis%40skills.tech&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"
2024-01-08 01:01:08,149:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.skills.demo%40newyorklife.com&reminder_type=eq.Welcome "HTTP/1.1 200 OK"
2024-01-08 01:01:08,240:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/reminders_tb?select=%2A&user_email=eq.skills.demo%40newyorklife.com&reminder_type=eq.Learning%20Progress%20Check-In "HTTP/1.1 200 OK"


[{'id': 59, 'user_email': 'luis@skills.tech', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T06:31:01+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False
[{'id': 60, 'user_email': 'skills.demo@newyorklife.com', 'reminder_type': 'Learning Progress Check-In', 'status': 'sent', 'scheduled_date': '2024-01-15T06:31:09+00:00', 'content': 'Learning Progress Check-In', 'sent_date': None, 'user_name': None}] False


In [2]:
from database.supa import supabase_user,supabase_admin  # Importar los clientes de Supabase
from supabase import create_client
from decouple import config

In [8]:
url_admin: str = config("SUPABASE_ADMIN_URL")
key_admin: str = config("SUPABASE_ADMIN_KEY")

supabase_admin_2 = create_client(supabase_url=url_admin,supabase_key= key_admin)
admin_data = supabase_admin.table("courses_tb").select("*").execute()
course_list=admin_data.data

for course in course_list:
    print(course["name"])
    print(course["id"])
    print(course["reference_files"])
    print(course["pdf_processed"])

2024-02-27 10:00:39,095:INFO - HTTP Request: GET https://hbvkehzhssiwpotkcxve.supabase.co/rest/v1/courses_tb?select=%2A "HTTP/1.1 200 OK"


Lean Analytics
350bc862-7d86-4bde-8ace-c2026aae277f
[{'0': '6c0bfedb-258a-4c77-9bad-b0e87c0d9c98/Lean Analytics'}]
False
Tec Course
6b21009c-98f0-4782-9510-496d0a830320
[{'0': '2e2aec92-0980-483e-9edd-05f187d10253/Tec_Course'}]
False
Commercial Real Estate for Beginners
78be68bd-c4f0-4690-b104-f9c744c19a51
[{'0': '11aafa3e-e3e9-43d3-aca9-f89388a66692/Commercial_Real_Estate_for_Beginners'}]
False
TC1028 Pensamiento computacional para ingenieria
38c62c87-322d-463f-8ee3-5f7da57b8af0
[{'0': '2e2aec92-0980-483e-9edd-05f187d10253/TC1028_Pensamiento_computacional_para_ingenieria'}]
False
Coursera: 1.- El Calculo - Modelo Lineal
081f021c-b4a2-4010-9aa9-16d1ea93fb3f
[{'0': '2e2aec92-0980-483e-9edd-05f187d10253/Coursera:_1.-_El_Calculo_-_Modelo_Lineal'}]
False
Enterprise AI
03373e3f-6f19-465a-a76b-7b88e11135c1
[{'0': '2f756050-f7f4-4764-a849-32bc7dea32ab/Enterprise_AI'}]
False
New York Life: Financial Advisor Development Program
7bc9e9e7-f1d2-4967-ad27-fd1697885f0b
[{'0': '47d455b9-760a-4d4a-b2b

In [ ]:
# Proyecto Usuario
url_user: str = config("SUPABASE_USER_URL")
key_user: str = config("SUPABASE_USER_KEY")
supabase_user = create_client(supabase_url=url_user,supabase_key= key_user)


response = supabase_user.table("responses_tb").select("*").eq("threadid", "4a37be7f-ce2c-4f19-aaaa-15f6d334a908").execute().data[0]
response

# Talents

In [2]:
from supabase import create_client
from decouple import config



from datetime import datetime, timedelta

# Obtener la fecha y hora actuales
now = datetime.now()

# Calcular la fecha de hace una semana
one_week_ago = now - timedelta(days=7)

url_supabase = config('SUPABASE_USER_URL')
key_supabase = config('SUPABASE_USER_KEY')
supabase = create_client(url_supabase, key_supabase)

members = supabase.table('members_tb').select("*").execute().data

2024-05-08 11:06:04,918:INFO - HTTP Request: GET https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/members_tb?select=%2A "HTTP/1.1 200 OK"


In [24]:
from datetime import datetime, timedelta, timezone

# Asegúrate de que 'one_week_ago' sea 'aware' utilizando UTC
one_week_ago = datetime.now(timezone.utc) - timedelta(days=7)

for member in members:
    print("---------------------------------------")
    print(f"ID: {member['id']}")
    print(f"Name: {member['name']}")
    print(f"Username: {member['username']}")

    # Intenta obtener los threads
    try:
        response = supabase.table('threads_tb').select("*").eq("memberid", member["id"]).execute()
        threads = response.data
        print(f"Threads fetched for member {member['name']}: {len(threads)}")
    except Exception as e:
        print(f"Error fetching threads for member {member['name']}: {str(e)}")
        threads = None

    # Contar cuántos threads tiene el miembro
    count_of_threads = len(threads) if threads else 0
    count_of_recent_threads = 0
    if threads:
        # Filtrar y contar los threads creados en la última semana
        try:
            recent_threads = [
                thread for thread in threads 
                if datetime.fromisoformat(thread["created_at"].replace("Z", "+00:00")) >= one_week_ago
            ]
            count_of_recent_threads = len(recent_threads)
            print(f"Recent threads for {member['name']}: {count_of_recent_threads}")
        except Exception as e:
            print(f"Error filtering recent threads for {member['name']}: {str(e)}")

    # Actualiza la columna 'talents' en 'members_tb'
    try:
        # Obtiene la columna actual 'talents', si no existe, se utiliza una lista vacía
        current_talents = member.get('talents', [])
        if count_of_threads > 5:
            if 1 not in current_talents:
                current_talents.append(1)
        if count_of_recent_threads > 1:
            if 2 not in current_talents:
                current_talents.append(2)

        # Actualiza la columna 'talents' con los nuevos valores
        if current_talents is not None:
            update_response = supabase.table('members_tb').update({
                'talents': [current_talents]  # Asegúrate de que esto sea un array
            }).eq('id', member['id']).execute()
            print(update_response)
        if current_talents is None:
            update_response = supabase.table('members_tb').update({
                'talents': []  # Asegúrate de que esto sea un array
            }).eq('id', member['id']).execute()
            print(update_response)

        print(f"Talents updated for {member['name']}: {current_talents}")
    except Exception as e:
        print(f"Error updating talents for {member['name']}: {str(e)}")

    # Imprimir el conteo total de threads y el conteo de threads recientes para el miembro
    print(f"{member['name']} tiene {count_of_threads} threads en total.")
    print(f"{member['name']} ha creado {count_of_recent_threads} threads en la última semana.")
    print("---------------------------------------")



---------------------------------------
ID: 69f3f1a3-626f-4541-bcb3-dc0155652b2f
Name: Ricardo Cevada
Username: RicardoNewYorkLife


2024-05-08 12:05:53,646:INFO - HTTP Request: GET https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/threads_tb?select=%2A&memberid=eq.69f3f1a3-626f-4541-bcb3-dc0155652b2f "HTTP/1.1 200 OK"


Threads fetched for member Ricardo Cevada: 19
Error filtering recent threads for Ricardo Cevada: Invalid isoformat string: '2024-03-01T21:22:27.17284+00:00'
Error updating talents for Ricardo Cevada: argument of type 'NoneType' is not iterable
Ricardo Cevada tiene 19 threads en total.
Ricardo Cevada ha creado 0 threads en la última semana.
---------------------------------------
---------------------------------------
ID: c4418ff4-0730-4557-82da-a7ca4147da24
Name: Ricardo Cevada
Username: RicardoBainAcademy


2024-05-08 12:05:53,944:INFO - HTTP Request: GET https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/threads_tb?select=%2A&memberid=eq.c4418ff4-0730-4557-82da-a7ca4147da24 "HTTP/1.1 200 OK"
2024-05-08 12:05:54,086:INFO - HTTP Request: GET https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/threads_tb?select=%2A&memberid=eq.f2871595-890f-46b3-9386-4cdc2942b4a3 "HTTP/1.1 200 OK"


Threads fetched for member Ricardo Cevada: 8
Error filtering recent threads for Ricardo Cevada: Invalid isoformat string: '2024-05-01T17:08:57.98452+00:00'
Error updating talents for Ricardo Cevada: argument of type 'NoneType' is not iterable
Ricardo Cevada tiene 8 threads en total.
Ricardo Cevada ha creado 0 threads en la última semana.
---------------------------------------
---------------------------------------
ID: f2871595-890f-46b3-9386-4cdc2942b4a3
Name: checho
Username: Checho.jh
Threads fetched for member checho: 0


...

Threads fetched for member Bruno Uriel Ortega Castrejón: 5
Error filtering recent threads for Bruno Uriel Ortega Castrejón: Invalid isoformat string: '2024-04-23T19:25:04.75227+00:00'
data=[{'id': '585b0bfc-4927-426b-bec9-91a2c7bfbc68', 'companyid': 'd54313a1-9c47-4d3d-b97a-1fdea82aef5d', 'username': 'BrunoU92', 'email': 'coub1908@gmail.com', 'name': 'Bruno Uriel Ortega Castrejón', 'ai_insights': None, 'created_at': '2024-04-23T18:03:45.485726+00:00', 'talents': [], 'active': True, 'last_login': '2024-05-08T15:57:17.807+00:00', 'score': None, 'onboarding': True, 'member_group': 'DCO', 'mbti': None, 'aditional_instructions': None, 'profilepicture': None}] count=None
Talents updated for Bruno Uriel Ortega Castrejón: None
Bruno Uriel Ortega Castrejón tiene 5 threads en total.
Bruno Uriel Ortega Castrejón ha creado 0 threads en la última semana.
---------------------------------------
---------------------------------------
ID: 7dff7761-57a1-45db-914e-181aaf0a56b3
Name: Elvia Ramirez
Usern

2024-05-08 12:06:05,810:INFO - HTTP Request: GET https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/threads_tb?select=%2A&memberid=eq.7dff7761-57a1-45db-914e-181aaf0a56b3 "HTTP/1.1 200 OK"
2024-05-08 12:06:05,951:INFO - HTTP Request: PATCH https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/members_tb?id=eq.7dff7761-57a1-45db-914e-181aaf0a56b3 "HTTP/1.1 200 OK"


Threads fetched for member Elvia Ramirez: 0
data=[{'id': '7dff7761-57a1-45db-914e-181aaf0a56b3', 'companyid': 'acb8bad3-aca4-48f5-abba-a21aa1fb1484', 'username': 'Admexus', 'email': 'contacto.admexus@gmail.com', 'name': 'Elvia Ramirez', 'ai_insights': None, 'created_at': '2024-04-26T16:04:11.549912+00:00', 'talents': [], 'active': True, 'last_login': None, 'score': None, 'onboarding': False, 'member_group': None, 'mbti': None, 'aditional_instructions': None, 'profilepicture': None}] count=None
Talents updated for Elvia Ramirez: None
Elvia Ramirez tiene 0 threads en total.
Elvia Ramirez ha creado 0 threads en la última semana.
---------------------------------------
---------------------------------------
ID: f0859289-b992-404e-be4d-b75e627434be
Name: Abimelec Anaya
Username: Abimelec


2024-05-08 12:06:06,083:INFO - HTTP Request: GET https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/threads_tb?select=%2A&memberid=eq.f0859289-b992-404e-be4d-b75e627434be "HTTP/1.1 200 OK"
2024-05-08 12:06:06,236:INFO - HTTP Request: PATCH https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/members_tb?id=eq.f0859289-b992-404e-be4d-b75e627434be "HTTP/1.1 200 OK"


Threads fetched for member Abimelec Anaya: 3
Error filtering recent threads for Abimelec Anaya: Invalid isoformat string: '2024-05-03T12:42:04.17194+00:00'
data=[{'id': 'f0859289-b992-404e-be4d-b75e627434be', 'companyid': '47d455b9-760a-4d4a-b2bd-9046704a2141', 'username': 'Abimelec', 'email': 'abimelec.demo@newyorklife.com', 'name': 'Abimelec Anaya', 'ai_insights': None, 'created_at': '2024-04-23T18:49:45.826698+00:00', 'talents': [], 'active': True, 'last_login': '2024-05-03T18:58:10.516+00:00', 'score': None, 'onboarding': False, 'member_group': None, 'mbti': None, 'aditional_instructions': None, 'profilepicture': True}] count=None
Talents updated for Abimelec Anaya: None
Abimelec Anaya tiene 3 threads en total.
Abimelec Anaya ha creado 0 threads en la última semana.
---------------------------------------


In [21]:
update_response = supabase.table('members_tb').update({'talents': []}).eq('id', "8b013804-faa6-426e-bfcc-43227f58e3c8").execute()
print(update_response)

2024-05-08 12:02:56,355:INFO - HTTP Request: PATCH https://hhtrmdrpowaaxlydfwjn.supabase.co/rest/v1/members_tb?id=eq.8b013804-faa6-426e-bfcc-43227f58e3c8 "HTTP/1.1 200 OK"


data=[{'id': '8b013804-faa6-426e-bfcc-43227f58e3c8', 'companyid': '6c0bfedb-258a-4c77-9bad-b0e87c0d9c98', 'username': 'ian333', 'email': 'sebastian@skills.tech', 'name': 'Ian Vazquez', 'ai_insights': None, 'created_at': '2024-01-24T02:18:13.229104+00:00', 'talents': [], 'active': True, 'last_login': '2024-04-25T21:49:58.599+00:00', 'score': 68, 'onboarding': None, 'member_group': None, 'mbti': None, 'aditional_instructions': None, 'profilepicture': None}] count=None
